In [1]:
import os
import numpy as np
import pandas as pd 
#import modin.pandas as pd
from tqdm import tqdm
import re
import datetime
from functools import partial
import gc

In [2]:
df_all_commiters = pd.read_csv("./commiters_emails.csv")

In [3]:
from bert_serving.client import BertClient
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])

array([[ 0.13186492,  0.3240412 , -0.82704383, ..., -0.37119597,
        -0.3925016 , -0.3172183 ],
       [ 0.24873525, -0.12334403, -0.38933882, ..., -0.44756222,
        -0.5591358 , -0.11345164],
       [ 0.28627354, -0.1858014 , -0.3090678 , ..., -0.29593664,
        -0.3931049 ,  0.07640221]], dtype=float32)

In [4]:
df_all_commiters.head()

,message_id,references,project_name,list_name,date,dealised_author_full_name,subject,body
0,<CA+FBdFQ7jRT+mr9Q-Ew=5k8Kc7zbeT3aMn5PoYLbTt9L...,<CAJjAHoMFvhT+xv21hh6r_Myxzk2vpRAhLLB_-K2eA8we...,superset,dev,2017-06-05 10:12:28,Ashutosh Chauhan,check -- -,yes list fri jun jim jagielski jim com write g...
1,<CA+FBdFRTx6k6EMebeb4jh_62Z4-LuU0a=tKAabBb2shv...,<CAHEEp7WxZztTjyZ0Tz+oPeHAPHPoPBZb71hoL4njtfg7...,superset,dev,2017-09-26 09:47:56,Ashutosh Chauhan,react apache superset,indeed great news ashutosh mon sep pm maxime b...
2,<JIRA.13130300.1515699055000.127061.1517866140...,<JIRA.13130300.1515699055000@Atlassian.JIRA> <...,superset,dev,2018-02-05 21:29:00,Ashutosh Chauhan,jira commented superset- send password creatio...,page com atlassian jira plugin system issuetab...
3,<BANLkTi=gMnmH146_FTiQH3ahQnnpeUp_8A@mail.gmai...,<BANLkTi=oq_QEwnwfwmcbrL8p9DdnCwObUw@mail.gmai...,flume,dev,2011-06-23 17:57:03,Aaron Kimball,plan moving mailing list apache,thu jun pm jonathan hsieh jon com write arg se...
4,<CAGfeOkdx-oTBnpeFhmmeTUZeSc_mofzO25jz20eVKOL+...,<CACAoQH2JT4uoOZGRe5dS=tUyQtG=VzFDqNcoEpvPb5RJ...,cordova,dev,2012-01-06 11:59:06,Dan Silivestru,doe anyone actually like jira email sent -dev ...,well fri jan gtanner com write get rid jira em...


In [5]:
#df_all_commiters["subject"].apply(lambda x: str(x)).values.tolist()

In [7]:
vector_subjects = bc.encode(df_all_commiters["subject"].apply(lambda x: str(x)).values.tolist())

/home/orzwang/.pyenv/versions/3.8.10/lib/python3.8/site-packages/bert_serving/client/__init__.py:294: UserWarning: some of your sentences have more tokens than "max_seq_len=30" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  warnings.warn('some of your sentences have more tokens than "max_seq_len=%d" set on the server, '


In [8]:
np.save("vector30_subjects.npy", vector_subjects)

In [3]:
vector_subjects = np.load("vector30_subjects.npy")

In [10]:
vector_subjects[:10]

array([[ 0.2048192 , -0.507999  ,  0.09057137, ..., -0.03647077,
        -0.3940924 ,  0.15549694],
       [-0.07200582, -0.23690899, -0.13029452, ..., -0.69624287,
        -0.33085585, -0.33923617],
       [-0.15763086, -0.46258152,  0.30271664, ..., -0.52064836,
        -0.2015468 ,  0.11546358],
       ...,
       [ 0.40586093, -0.5096089 ,  0.66622514, ...,  0.00303813,
         0.12995748,  0.52954125],
       [-0.19023953, -0.6462398 ,  0.56429684, ..., -0.48529845,
        -0.07989247,  0.18042935],
       [ 0.20377992, -0.8245825 , -0.02991275, ..., -0.16454342,
        -0.09343997, -0.24743496]], dtype=float32)

In [5]:
vector_bodies = np.load("vector200_bodies.npy")

In [6]:
gc.collect()

22

In [7]:
df_all_commiters["vector_subjects"] = pd.Series(vector_subjects)
df_all_commiters["vector_bodies"] = pd.Series(vector_bodies)

ValueError: Data must be 1-dimensional

In [8]:
df_vector_subjects = pd.DataFrame(vector_subjects)
df_vector_bodies = pd.DataFrame(vector_bodies)

In [9]:
df_vector_subjects

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.204819,-0.507999,0.090571,-0.448471,0.375577,-0.221125,0.007410,0.691307,-0.158918,0.013177,...,0.466173,-0.030598,-0.171947,-0.451510,-0.211386,0.110515,-0.088458,-0.036471,-0.394092,0.155497
1,-0.072006,-0.236909,-0.130295,0.084533,0.208961,-0.439237,0.041204,0.293959,-0.357971,-0.062890,...,0.236655,-0.058175,-0.196468,-0.266063,0.240864,0.433130,-0.019434,-0.696243,-0.330856,-0.339236
2,-0.157631,-0.462582,0.302717,-0.008316,0.548475,0.233083,0.144578,0.340630,-0.012533,0.168474,...,0.264887,0.005636,-0.230705,-0.184273,0.388936,0.176506,-0.362768,-0.520648,-0.201547,0.115464
3,0.111624,-0.416838,0.358992,-0.006547,0.194561,0.044064,-0.113744,0.506783,-0.701127,0.062781,...,0.500622,-0.084329,-0.389650,-0.183381,0.725766,0.388576,-0.239179,-0.729360,0.000356,-0.048344
4,0.187182,-0.125304,0.454186,0.056435,-0.107367,-0.120712,0.018339,0.852625,-0.338909,0.087598,...,0.224909,-0.186749,-0.355919,-0.315692,0.410230,0.183412,-0.537006,-0.631323,0.261165,0.047418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121258,0.173959,0.000439,0.327966,-0.009223,0.343574,-0.315566,0.166076,-0.028029,-0.380087,-0.426212,...,0.072788,-0.179679,-0.300810,-0.450498,0.255854,0.313473,0.071860,-0.618623,-0.371089,-0.200543
1121259,0.173959,0.000439,0.327966,-0.009223,0.343574,-0.315566,0.166076,-0.028029,-0.380087,-0.426212,...,0.072788,-0.179679,-0.300810,-0.450498,0.255854,0.313473,0.071860,-0.618623,-0.371089,-0.200543
1121260,-0.032514,-0.501741,-0.063646,-0.093875,0.882153,-0.073044,0.251120,0.277988,-0.009285,-0.160256,...,0.489254,0.036903,-0.490298,-0.504290,0.411811,0.611617,-0.277818,-0.682317,-0.255431,-0.122456
1121261,0.050131,-0.097492,-0.184836,-0.178275,0.292156,0.266386,-0.081752,0.236426,-0.197655,-0.343586,...,0.191903,-0.376965,0.360123,-0.183666,-0.137396,-0.274581,-0.055172,-0.745906,-0.327875,0.048705


In [10]:
df_vector_bodies

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.042233,0.468242,0.753363,-0.311905,0.157820,-0.195879,0.257697,0.366997,-0.010694,-0.510621,...,-0.295695,-0.416872,-0.047434,-0.012088,0.319048,0.156520,-0.939255,-0.462316,-0.145303,-0.046418
1,0.021487,0.005634,0.502711,-0.086015,0.572555,-0.679561,0.077415,0.214838,-0.063668,-0.444987,...,-0.177617,-0.137732,-0.142966,-0.214508,0.347240,0.344018,-0.612595,-0.405896,-0.361411,-0.265411
2,-0.250653,0.181648,0.627695,-0.286776,0.364591,-0.276215,0.124130,0.196918,-0.232846,-0.222329,...,-0.196546,-0.256780,0.068282,-0.112549,0.280718,0.344885,-0.807309,-0.467994,-0.435828,0.011934
3,-0.117559,0.267312,0.800553,-0.357053,0.315310,-0.348842,0.008527,0.240120,-0.289082,-0.312852,...,-0.284726,-0.504100,-0.034715,-0.148827,0.135251,-0.143713,-0.721749,-0.344756,-0.238567,-0.144181
4,-0.237711,0.159207,0.812839,-0.208513,-0.057939,-0.144360,0.189074,0.377302,-0.545933,-0.363685,...,-0.245452,-0.362685,0.155246,-0.077833,0.313194,0.161798,-0.768129,-0.453839,-0.112421,-0.029235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121258,-0.227035,0.259295,0.749243,-0.338304,0.300313,-0.418559,0.227987,0.267221,0.213315,-0.255476,...,-0.230585,-0.504264,-0.009978,0.142878,0.115418,0.118878,-0.622369,-0.317193,-0.478412,-0.112514
1121259,-0.186059,0.293233,0.639381,-0.293910,0.360155,-0.351132,0.188732,0.211933,0.246311,-0.211495,...,-0.225327,-0.543448,0.033016,0.058136,0.113196,0.126201,-0.727568,-0.283272,-0.450673,-0.089302
1121260,-0.086909,0.072472,0.568935,-0.390131,0.283059,-0.385017,0.342535,0.177906,0.035953,-0.257448,...,0.052216,-0.103411,0.044908,-0.404456,0.519645,0.429317,-0.552868,-0.606797,-0.461828,0.046932
1121261,-0.215436,0.256646,0.551269,-0.457037,0.442454,-0.180838,0.211365,-0.168178,-0.057999,-0.635327,...,-0.058901,-0.384406,0.301779,-0.005706,0.067218,0.426234,-0.722766,-0.596688,-0.422455,-0.173967


In [12]:
from sklearn.cluster import DBSCAN

X = df_vector_subjects[df_vector_subjects.columns]

db = DBSCAN(eps=10, min_samples=20).fit(X)

labels = db.labels_
df_all_commiters["labels"] = labels

: 

: 

In [ ]:
from sklearn import metrics 

score = metrics.silhouette_score(X,df_all_commiters.labels) 
print(score)